(sec:regex)=
# 正規表現の基礎

本節では、正規表現の基本について学ぶ。

## 正規表現とは？

正規表現 (regular expression)とは、特定の文字パターンに合致する文字列を見つける仕組みを指す。

In [1]:
import re

In [3]:
text = "Hello, world!"
pat = re.compile("Hello")
match = pat.search(text)
if match is not None:
    print(f'Matched text is "{match.group(0):s}"')
else:
    print("Pattern not matched.")

Matched text is "Hello"


正規表現は大文字と小文字を区別するので、`re.compile("hello")`のように`pat`を定義すると、マッチしなくなる。大文字小文字を区別せずに文字列のマッチを検索する方法もあるので後述する。

正規表現のパターン文字列である`pat`には指定した文字列の中に正規表現とマッチする文字列が存在するかを調べる`search`の他にも

- 文字列の先頭から見て正規表現とマッチするかを調べる `match`
- 文字列が正規表現と完全にマッチするかを調べる `fullmatch`
- マッチした文字列を別の文字列で置き換える `sub`

などのメソッドが用意されている。

**matchの使用例**

In [4]:
text = "Hello, world!"
pat = re.compile("ello")
if pat.match(text) is not None:
    print("Pattern found!")
else:
    print("Pattern not found!")

Pattern not found!


In [5]:
text = "Hello, world!"
pat = re.compile("Hello")
if pat.match(text) is not None:
    print("Pattern found!")
else:
    print("Pattern not found!")

Pattern found!


**fullmatch**の使用例

In [6]:
text = "Hello, world!"
pat = re.compile("Hello, world")
if pat.fullmatch(text) is not None:
    print("Pattern matches perfectly!")
else:
    print("Pattern does not match!")

Pattern does not match!


In [7]:
text = "Hello, world!"
pat = re.compile("Hello, world!")
if pat.fullmatch(text) is not None:
    print("Pattern matches perfectly!")
else:
    print("Pattern does not match!")

Pattern matches perfectly!


**subの使用例**

In [8]:
text = "Hello, world!"
pat = re.compile("world")
text2 = pat.sub("Japan", text)
print(text2)

Hello, Japan!


```{admonition} 正規表現の基本
:class: note

- 正規表現は特定の文字パターンに合致する文字列を探すのに用いる
- Pythonにおいては標準ライブラリの`re`を用いる
- `re.compile`で文字パターンを定義した後、`search`や`fullmatch`を使って文字列を解析する
```

## 正規表現と決定性有限オートマトン

## 正規表現のシンタックス 初級編

正規表現は特定の文字列を探すだけでなく、より複雑なパターンを持った文字列を探すことができる。

### 何らかの1文字

どこかに1文字、何でも良いので文字が存在することを表したい場合には、`a.c`のように`.`を用いる。この正規表現は例えば`adc`や`aBc`など、`.`の部分に何らか1文字がはいるような文字列とマッチするが、`a.c`のように`.`に対応する場所に2文字以上ある場合にはマッチしない。

In [3]:
pat = re.compile("a.c")
print(pat)
print("adc :", pat.fullmatch("adc") is not None)
print("aBc :", pat.fullmatch("aBc") is not None)
print("adBc :", pat.fullmatch("adBc") is not None)

re.compile('a.c')
adc : True
aBc : True
adBc : False


### 文字の繰り返し

a, b, cの3文字だけからなる単語が存在するかどうかを調べたい場合には、文字の繰り返しを表す`*`あるいは`+`を用いる。例えば、`aaabcccc`のような文字列とマッチさせたい場合`a+bc+`のように繰り返したい文字の後に`+`あるいは`*`をつける。

In [4]:
text = "aaabcccc"
pat = re.compile("a+bc+")
print(pat)
print(text, ":", pat.fullmatch(text) is not None)

re.compile('a+bc+')
aaabcccc : True


同じ文字の繰り返しを表す`+`と`*`は、その繰り返し回数の見方に違いがある。`+`は直前の文字が**1回以上繰り返す**場合にのみマッチするのに対し、`*`は**0回以上の繰り返し**にもマッチする。従って、`ab+`というパターンは`a`にはマッチせず、`ab`にはマッチする一方で、`ab*`というパターンは`a`にマッチし、なおかつ`ab`にもマッチする。

In [5]:
text1 = "a"
text2 = "ab"

In [6]:
pat = re.compile("ab+")
print(pat)
print(text1, ":", pat.fullmatch(text1) is not None)
print(text2, ":", pat.fullmatch(text2) is not None)

re.compile('ab+')
a : False
ab : True


In [7]:
pat = re.compile("ab*")
print(pat)
print(text1, ":", pat.fullmatch(text1) is not None)
print(text2, ":", pat.fullmatch(text2) is not None)

re.compile('ab*')
a : True
ab : True


加えて、回数を制限した文字の繰り返しを定義することもできる。例えば、`abbccc`は検出したいが、`c`が4回以上繰り返す場合や`c`が含まれない場合は除外したいという場合には、繰り返し回数は1-3回に指定して、`abc{1,3}`のような文字パターンを定義することができる。

In [8]:
pat = re.compile("abc{1,3}")
print("abcc :", pat.fullmatch("abcc") is not None)
print("abcccc :", pat.fullmatch("abcccc") is not None)

abcc : True
abcccc : False


### 文字セット

ここまでの例では、`+`や`*`といった文字の繰り返しは、特定の1文字にしか適用していなかった。しかし、Pythonに限らず、正規表現には特定の範囲の文字を扱う**文字セット**が用意されている。

例えば、`[a-z]`のように書くと、小文字のアルファベット全てにマッチする文字パターンを表すことができる。これに`+`や`*`を組み合わせると小文字アルファベットだけで構成されている単語かどうかをチェックすることができる。

In [9]:
pat = re.compile("[a-z]+")
print("Hello : ", pat.fullmatch("Hello") is not None)
print("hello : ", pat.fullmatch("hello") is not None)

Hello :  False
hello :  True


また、Pythonの正規表現ではASCIIコード表で連続する文字種を文字セットの始点と終点に指定することができるため、`[x-z]`のように書くと、小文字でx, y, zのいずれかにマッチするような文字パターンを作ることもできる。

文字セットを表す`[...]`の内側には複数の文字や文字の範囲を含めることができ、例えば、大文字、小文字の全てのアルファベットと、0-9の数字を含めたい場合、`[a-zA-Z0-9]`のような文字パターンを作ることができる。これに加えて、さらに、ハイフン (`-`)やスペース (` `)を含めたい場合には `[a-zA-Z0-9- ]`のように書けばよい。

In [10]:
text = "2-1 Naka Kunitachi-shi"
pat = re.compile("[a-zA-Z0-9- ]+")
print(text, ":", pat.fullmatch(text) is not None)

2-1 Naka Kunitachi-shi : True


### 最短マッチ、最長マッチ

### 接頭辞、接尾辞

### エスケープ文字

## 正規表現のシンタックス 中級編

### 検索方法の詳細設定

### サブグループ

上記の例では、マッチした文字列を取り出すために `match.group(0)`のように`group`メソッドに`0`を指定した。この`group`はマッチした文字列内にあるサブグループのことで、`0`はマッチした文章全体を示す。また`match.group(0)`は`match[0]`とも書くことができる。

### 名前付きサブグループ

### 日本語の取り扱い